In [6]:
# simple code for chat with web 

In [7]:
# Import of the packages

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List

In [2]:
#aded prompts for templates
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

In [3]:
# aded acces to the web via langchinabs

In [4]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# add parsers

In [6]:
from langchain.output_parsers import PydanticOutputParser

In [7]:
# load env and vriables
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [8]:
# main function

In [9]:
def ask_chat(model_name, template, variables, parser):
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=model_name, temperature=0.7)
    message = HumanMessagePromptTemplate.from_template(template=template)
    prompt = ChatPromptTemplate.from_messages(messages=[message])
    # Formatting the prompt with the given variables
    prompt_with_values = prompt.format_prompt(**variables) 
    # creation of messge
    messages = prompt_with_values.to_messages()
    answer = llm(messages)
    output = parser.parse(answer.content)
    
    return output

In [10]:
# function for accessing the web

In [11]:
def get_web_content(url):
    loader = WebBaseLoader(url)
    document = loader.load()
    return document

In [12]:
# create template

In [13]:
PROMPT_TEMPLATE = """
    Get the information about the webpage statistics from the table from this {webpage_content}. Put them in the order
    today, yesterday, at the average at today, at the average at yesterday. ignore number of diferences
    {output_format}
    """

In [14]:
#create output report / from gpt promt for simplicity 

In [15]:
class Metrics(BaseModel):
    views: List[int] = Field(..., description="Number of views for each period.")
    sessions: List[int] = Field(..., description="Number of sessions for each period.")
    visitors: List[int] = Field(..., description="Number of unique visitors for each period.")
    hosts: List[int] = Field(..., description="Number of hosts for each period.")
    reloads: List[int] = Field(..., description="Number of page reloads for each period.")
    average_online: List[int] = Field(..., description="Average number of users online for each period.")
    average_active_online: List[int] = Field(..., description="Average number of active users online for each period.")
    average_duration: List[float] = Field(..., description="Average duration of sessions in minutes for each period.")
    views_per_visitor: List[float] = Field(..., description="Average number of views per visitor for each period.")

    def __str__(self):
        msgs = [f"{k}: \t {v}" for k,v in self.__dict__.items()]
        return '\n'.join(msgs)
      

In [16]:
# get content

In [17]:
url = "https://www.liveinternet.ru/stat/rg.ru/index.html"

In [18]:
web_content = get_web_content(url)

In [19]:
content_to_summarize = " ".join([doc.page_content for doc in web_content])

In [20]:
# creation of the parser

In [21]:
parser = PydanticOutputParser(pydantic_object=Metrics)

In [22]:
variables = {"webpage_content": content_to_summarize,
            "output_format": parser.get_format_instructions()}

In [23]:
answer =  ask_chat(model_name="gpt-4o",  template=PROMPT_TEMPLATE, variables=variables, parser=parser)

C:\Users\micha\PycharmProjects\AIPlayground\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [24]:
print(answer)

views: 	 [14364, 65945, 63818, 65823]
sessions: 	 [12778, 59811, 57695, 59588]
visitors: 	 [12613, 57655, 55330, 57255]
hosts: 	 [12186, 52834, 50089, 51976]
reloads: 	 [35, 87, 88, 93]
average_online: 	 [347, 633, 611, 631]
average_active_online: 	 [31, 51, 53, 52]
average_duration: 	 [0.23, 0.22, 0.25, 0.24]
views_per_visitor: 	 [1.1, 1.1, 1.1, 1.1]
